# Differential gene expression analysis - code example for edgeR (pseudobulk)
Using the data set Misharin.

In [1]:
#install.packages("anndata")
library(anndata)

In [12]:
file_name <- 'home/sch/schonner/MaPra/schiller_for_diffEx_edgeR.h5ad'
adata_pb <- read_h5ad(file_name)
adata_pb   # file with TypeErrors while saving

ERROR: Error in py_call_impl(callable, dots$args, dots$keywords): FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'home/sch/schonner/MaPra/schiller_for_diffEx_edgeR.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)



In [15]:
adata_test <- read_h5ad('/home/sch/schonner/MaPra/schiller_for_diffEx_edgeR_test.h5ad')
adata_pb   # file with conversions

AnnData object with n_obs × n_vars = 134 × 18031
    obs: 'condition', 'manual_celltype_annotation', 'batch', 'sample', 'lib_size', 'log_lib_size'
    uns: 'batch_colors', 'condition_colors', 'log1p', 'manual_celltype_annotation_colors', 'pca', 'sample_colors'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'counts'

In [16]:
adata_test.obs

ERROR: Error in eval(expr, envir, enclos): object 'adata_test.obs' not found


## 1. Preparing some functions

### Separate function to fit an edgeR GLM:

In [10]:
fit_model <- function(adata_){
    # create an edgeR object with counts and grouping factor
    y <- DGEList(assay(adata_, "X"), group = colData(adata_)$label)
    # filter out genes with low counts
    print("Dimensions before subsetting:")
    print(dim(y))
    print("")
    keep <- filterByExpr(y)
    y <- y[keep, , keep.lib.sizes=FALSE]
    print("Dimensions after subsetting:")
    print(dim(y))
    print("")
    # normalize
    y <- calcNormFactors(y)
    # create a vector that is concatentation of condition and cell type that we will later use with contrasts
    group <- paste0(colData(adata_)$label, ".", colData(adata_)$cell_type)
    replicate <- colData(adata_)$replicate
    # create a design matrix: here we have multiple donors so also consider that in the design matrix
    design <- model.matrix(~ 0 + group + replicate)
    # estimate dispersion
    y <- estimateDisp(y, design = design)
    # fit the model
    fit <- glmQLFit(y, design)
    return(list("fit"=fit, "design"=design, "y"=y))
}

## 2. Pseudobulk

### 2.1 One group

In [11]:
adata_AT1 <- adata_pb[adata_pb.obs["manual_celltype_annotation"] == "AT1"]
adata_AT1

ERROR: Error in .process_index(oidx, nrow(x)): object 'adata_pb.obs' not found


In [23]:
adata_AT1.obs_names = [
    name.split("_")[2] + "_" + name.split("_")[3] for name in adata_AT1.obs_names
]

In [33]:
outs <- fit_model(adata_AT1)

NotImplementedError: Conversion 'py2rpy' not defined for objects of type '<class 'numpy.ndarray'>'

In [ ]:
fit <- outs$fit
y <- outs$y

In [ ]:
plotMDS(y, col=ifelse(y$samples$group == "stim", "red", "blue"))

In [51]:
plotBCV(y)


Error in is(y, "DGEList") : object 'y' not found


RInterpreterError: Failed to parse and evaluate line 'plotBCV(y)\n'.
R error message: 'Error in is(y, "DGEList") : object \'y\' not found'

In [ ]:
myContrast <- makeContrasts('groupstim.AT1-groupctrl.AT1', levels = y$design)
qlf <- glmQLFTest(fit, contrast=myContrast)
# get all of the DE genes and calculate Benjamini-Hochberg adjusted FDR
tt <- topTags(qlf, n = Inf)
tt <- tt$table

In [ ]:
tr <- glmTreat(fit, contrast=myContrast, lfc=1.5)
print(head(topTags(tr)))

In [ ]:
plotSmear(qlf, de.tags = rownames(tt)[which(tt$FDR<0.01)])

### 2.2 Multiple groups